In [1]:
import json
import numpy as np
from tqdm import tqdm
from collections import Counter

In [2]:
from gensim.models import word2vec
from gensim import models
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from lexrank import LexRank

In [3]:
pre_data_PATH = './data/'

In [4]:
with open(pre_data_PATH+'com_product_intro_word_articles_test.json') as jsonfile:
    com_product_intro_word = json.load(jsonfile)
com_product_intro_word

[['椰林 遍佈 島 美托三角洲 離 胡志明市 段 距離 但 許多 遊客 深受 那裡 水鄉澤國 傳統 集市 的 吸引 願意 驅車 前往 ',
  '胡志明市 出發 便利 接送 服務 你 少 交通 的 煩惱 ',
  '我們 搭上 小 船 入坐 馬車 到 濃郁 風情 湄公河 市鎮 一探 吧 ',
  '來到 越南 胡志明市 預訂 印度 支那號 遊船 晚宴 共有 種 套餐 供 選擇 讓 在 欣賞 西貢河 沿岸 夜景 之餘 放鬆 享用 精緻 晚宴 美食 遊船 更 精彩 舞蹈 民謠 音樂 傳統 表演 帶給 胡志明 浪漫 夜晚 特別 體驗 ',
  '胡志明 古芝 地道 私人 小團 日 遊 搭乘 舒適 空調 專車 前往 越南 戰爭 時期 重要 遺跡 全程 專業 中文 英語 導遊 隨行 導覽 帶 仔細 認識 古芝 地道 歷史 背景 適合 人 一同 搭 車 出遊 親身 感受 越戰 時期 緊張 氣氛 ',
  '越南  Viettel  上網 電話卡 前  8 GB  高速 上網 流量 超過 後 降速  64 kbps 效期  15 天 再 加贈 免費 接聽 來電 適用 越南 全 境 河內 胡志明市 重點 觀光區 皆 使用 ， 方便 旅途 中 隨時 卡 上網 ',
  '想 越南 走跳 能 少 最 便宜 最 方便 背包客 必備 交通 方式  Open Bus ',
  '事先 預定 巴士 的 座位 不用 害怕 熱門 時段 不了 車 '],
 ['個 小時 穿越 泰國 古今 了 解 泰國 豐厚 迷人 歷史 文化 ',
  '專業 演員 配合 絢爛 舞台 效果 曼谷暹羅 天使 劇場 你 重新 認識 泰國 ',
  '還可以 開場 前 欣賞 戶外 表演 參觀 民俗 展區 享用 自助 晚餐 品嘗 道地 泰國 美食 ',
  '專業 貼心 中文 導遊 陪伴 下 體驗 傳統 市集 鐵道 共存 奇景 美功 鐵道 市集 暢遊 百 年 歷史 丹嫩莎朵 水 市場 看看 式 特色 小吃 個性 商店 安帕瓦 水上 市場 參觀 獨具 特色 百 年 基督教堂 參訪 泰國 十 靈廟 一 「 樹中廟 」 並 唯美 夜色 中 迎 微風 ， 乘船 觀賞 閃爍 飛舞 螢火蟲 ',
  'Tripadvisor 顆 星 滿分 評價 曼谷泰式 烹飪 教室 乘船 觀賞 昭披 耶河 風光 後 抵達 綠意盎然 私人 香草 花園 

## LexRank

### Build Graph

In [7]:
similarity_matrix_list = []
for i in tqdm(range(len(com_product_intro_word))):
    N = 20       #set up the length limit
    sent_num  = len(com_product_intro_word[i])
    lxr = LexRank(com_product_intro_word)
    tf_scores = [Counter(com_product_intro_word[i][j].split()) for j in range(len(com_product_intro_word[i])) if j < sent_num]
    similarity_matrix = lxr._calculate_similarity_matrix(tf_scores)
    similarity_matrix_new = np.pad(array=similarity_matrix, pad_width=((0, N - sent_num), (0, N - sent_num)), mode='constant', constant_values=0)
    print(similarity_matrix_new)
    for j in range(len(similarity_matrix_new)):
        similarity_matrix_new[j][j] = 0
    similarity_matrix_list.append(similarity_matrix_new)

  1%|          | 1/174 [00:02<05:51,  2.03s/it]

[[1.         0.15122737 0.         0.05359518 0.00109387 0.04450084
  0.         0.00222882 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.15122737 1.         0.         0.14261965 0.         0.13448279
  0.15818187 0.00673554 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         1.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.05359518 0.14261965 0.         1.         0.09231041 0.10214837
  0.05091473 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.00109387 0.         0.         0.09231041 1.         0.05200761
  0.04476923 0.01387705 0.         0.         0.         0.
  0.         0.         0.  

  1%|          | 2/174 [00:04<05:52,  2.05s/it]

[[1.00000000e+00 1.40500502e-01 1.62508397e-01 7.88785069e-02
  9.07863254e-02 2.64359458e-02 0.00000000e+00 0.00000000e+00
  1.49528544e-02 3.00747655e-01 1.27178200e-01 1.32420195e-01
  1.40337187e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.40500502e-01 1.00000000e+00 6.69846664e-02 3.20618946e-02
  3.74214000e-02 2.32831208e-02 0.00000000e+00 2.41191674e-03
  0.00000000e+00 1.23965787e-01 5.24218409e-02 5.45825493e-02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.62508397e-01 6.69846664e-02 1.00000000e+00 4.36839625e-02
  4.32830605e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.43383697e-01 6.06331594e-02 6.31323196e-02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [7.88785069e-02 3.20618946e-02 4.36839625e-02 1.00000000e+00
  3.6

  2%|▏         | 3/174 [00:06<05:50,  2.05s/it]

[[1.         0.25560817 0.12065026 0.         0.24752985 0.08259409
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.25560817 1.         0.24116941 0.00862795 0.48112553 0.20434656
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.12065026 0.24116941 1.         0.26883062 0.21008547 0.16486779
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.00862795 0.26883062 1.         0.00843659 0.17633683
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.24752985 0.48112553 0.21008547 0.00843659 1.         0.12305718
  0.         0.         0.         0.         0.         0.
  0.         0.     

  2%|▏         | 3/174 [00:08<07:50,  2.75s/it]


ValueError: index can't contain negative values

In [ ]:
ranked_sentence_list = []
for i in tqdm(range(len(com_product_intro_word))):
    lex_scores = lxr.rank_sentences(com_product_intro_word[i], threshold=None)
    zipped = zip(lex_scores, com_product_intro_word[i])
    ranked_sentence = sorted(zipped, key = lambda x: x[0], reverse = True)
    ranked_sentence_list.append(ranked_sentence)
ranked_sentence_list

### Sentence Selection

In [ ]:
from functools import reduce
num = 6 # choose top n sentences
sentence_select_list = []
for s in ranked_sentence_list:
    l = min(len(s),num)
    sentence_select = [y for (x,y) in s[0:l]]
    
    sentence_select = reduce(lambda x, y:str(x)+str(y),sentence_select)
    sentence_select_list.append(sentence_select)

In [ ]:
result_PATH = './data/result/'
with open(result_PATH +'lexrank.json', 'w') as outfile:
    json.dump(sentence_select_list, outfile)

In [ ]:
sentence_select_list